In [4]:
#pbject/dict as a series
import pandas as pd

cal = {'day1':420 ,'day2':520 ,'day3':600}

var = pd.Series(cal)

var

day1    420
day2    520
day3    600
dtype: int64

In [17]:
var.day1

420

In [12]:
#df from dict
dataset = {
    'cars':['volvo','lexus','toyota'],
    'passings' :[2,3,7]}
dset = pd.DataFrame(dataset)
dset

,cars,passings
0,volvo,2
1,lexus,3
2,toyota,7


In [11]:
#rows of var
dset.cars

0     volvo
1     lexus
2    toyota
Name: cars, dtype: object

In [9]:
#list as a series
a = [1,7,2]
myvar = pd.Series(a)
myvar

0    1
1    7
2    2
dtype: int64

In [27]:
import numpy as np
import tensorflow as tf

In [64]:
dataset = pd.read_pickle('/content/drive/MyDrive/carpricesdata.pkl')

# Separate Target Variable and Predictor Variables
TargetVariable=['Price']
Predictors=['Age', 'KM', 'Weight', 'HP', 'MetColor', 'CC', 'Doors']

X=dataset[Predictors].values
y=dataset[TargetVariable].values

### Sandardization of data ###
from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)
TargetVarScalerFit=TargetVarScaler.fit(y)

# Generating the standardized values of X and y
X=PredictorScalerFit.transform(X)
y=TargetVarScalerFit.transform(y)

# Split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Quick sanity check with the shapes of Training and testing datasets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1004, 7)
(1004, 1)
(431, 7)
(431, 1)


In [65]:
# importing the libraries
from keras.models import Sequential
from keras.layers import Dense

# create ANN model
model = Sequential()

# Defining the Input layer and FIRST hidden layer, both are same!
model.add(Dense(units=5, input_dim=7, kernel_initializer='normal', activation='relu'))

# Defining the Second layer of the model
# after the first layer we don't have to specify input_dim as keras configure it automatically
model.add(Dense(units=5, kernel_initializer='normal', activation='tanh'))

# The output neuron is a single fully connected node 
# Since we will be predicting a single number
model.add(Dense(1, kernel_initializer='normal'))

# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Fitting the ANN to the Training set
model.fit(X_train, y_train ,batch_size = 20, epochs = 50, verbose=1)

Epoch 1/50
51/51 [==============================] - 2s 5ms/step - loss: 0.9800
Epoch 2/50
51/51 [==============================] - 0s 6ms/step - loss: 0.9198
Epoch 3/50
51/51 [==============================] - 0s 5ms/step - loss: 0.6807
Epoch 4/50
51/51 [==============================] - 0s 6ms/step - loss: 0.4017
Epoch 5/50
51/51 [==============================] - 0s 4ms/step - loss: 0.2776
Epoch 6/50
51/51 [==============================] - 0s 4ms/step - loss: 0.2357
Epoch 7/50
51/51 [==============================] - 0s 6ms/step - loss: 0.2127
Epoch 8/50
51/51 [==============================] - 0s 4ms/step - loss: 0.1956
Epoch 9/50
51/51 [==============================] - 0s 5ms/step - loss: 0.1829
Epoch 10/50
51/51 [==============================] - 0s 2ms/step - loss: 0.1729
Epoch 11/50
51/51 [==============================] - 0s 2ms/step - loss: 0.1656
Epoch 12/50
51/51 [==============================] - 0s 2ms/step - loss: 0.1599
Epoch 13/50
51/51 [==============================

In [66]:
# Defining a function to find the best parameters for ANN
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[5, 10, 15, 20]
    epoch_list  =   [5, 10, 50, 100]
    
    import pandas as pd
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'Accuracy'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            # create ANN model
            model = Sequential()
            # Defining the first layer of the model
            model.add(Dense(units=5, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))

            # Defining the Second layer of the model
            model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))

            # The output neuron is a single fully connected node 
            # Since we will be predicting a single number
            model.add(Dense(1, kernel_initializer='normal'))

            # Compiling the model
            model.compile(loss='mean_squared_error', optimizer='adam')

            # Fitting the ANN to the Training set
            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)

            MAPE = np.mean(100 * (np.abs(y_test-model.predict(X_test))/y_test))
            
            # printing the results of the current iteration
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'Accuracy:', 100-MAPE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), 100-MAPE]],
                                                                    columns=['TrialNumber', 'Parameters', 'Accuracy'] ))
    return(SearchResultsData)


######################################################
# Calling the function
ResultsData=FunctionFindBestParams(X_train, y_train, X_test, y_test)

14/14 [==============================] - 0s 2ms/step
1 Parameters: batch_size: 5 - epochs: 5 Accuracy: 42.67375765491029
14/14 [==============================] - 0s 1ms/step
2 Parameters: batch_size: 5 - epochs: 10 Accuracy: 45.53282794796345
14/14 [==============================] - 0s 2ms/step
3 Parameters: batch_size: 5 - epochs: 50 Accuracy: 53.71710522692574
14/14 [==============================] - 0s 2ms/step
4 Parameters: batch_size: 5 - epochs: 100 Accuracy: 59.42545869885021
14/14 [==============================] - 0s 3ms/step
5 Parameters: batch_size: 10 - epochs: 5 Accuracy: 30.109460417602705
14/14 [==============================] - 0s 2ms/step
6 Parameters: batch_size: 10 - epochs: 10 Accuracy: 33.62589503996088
14/14 [==============================] - 0s 2ms/step
7 Parameters: batch_size: 10 - epochs: 50 Accuracy: 60.50203999631439
14/14 [==============================] - 0s 2ms/step
8 Parameters: batch_size: 10 - epochs: 100 Accuracy: 60.110828210350334
14/14 [===========